In [1]:
# ! pip install hazm
# ! pip install gensim

In [2]:
import pandas as pd
import numpy as np
import re
from __future__ import unicode_literals
import hazm
import nltk
import codecs
import tqdm
import gensim

In [3]:
with open('masnavi.txt', 'r', encoding="utf8") as infile:
    masnavi = infile.readlines()

In [4]:
data = []

In [5]:
def process_beyt(text):
    result = re.search("(\d{1,3})\.(\d{1,3})", text)
    if result:
        poem, beyt = result.groups()
        beyt_text = [mesra for mesra in re.sub("(\d{1,3})\.(\d{1,3})", "", text).split("\t") if mesra]
        data.append((section, poem, beyt, beyt_text[0] + ' ' + beyt_text[1], beyt_text[0], beyt_text[1]))
    return None

In [6]:
section = 0
for line in masnavi:
    if re.search("(?:دفتر).*.(?:مثنوی)", line):
        section += 1
        if section == 7:
            break
        print(f"Processing Dafter {section}")
    else:
        process_beyt(line)

Processing Dafter 1
Processing Dafter 2
Processing Dafter 3
Processing Dafter 4
Processing Dafter 5
Processing Dafter 6


In [7]:
masnavi_df = pd.DataFrame(data, columns=['Daftar', 'Poem', 'BNo', 'B', 'M1', 'M2'])

In [8]:
lemmatizer = hazm.Lemmatizer()

In [9]:
normalizer = hazm.Normalizer()

In [10]:
def remove_new_line_char(series):
    return series.replace("\n",'')

In [11]:
stopwords = [normalizer.normalize(x.strip()) for x in codecs.open('stopwords.txt','r','utf-8').readlines()]

In [12]:
persian_punctuation = ['،','؟',':','*','«',"»"]

In [13]:
clean_text = lambda x: [t for t in x if t not in stopwords + persian_punctuation]

In [14]:
masnavi_df['B'] =  masnavi_df['B'].apply(lambda x:lemmatizer.lemmatize(normalizer.normalize(x)))

In [15]:
masnavi_df['B_tokenized'] = masnavi_df['B'].apply(lambda x:hazm.word_tokenize(x))

In [16]:
masnavi_df['B_tokenized'] = masnavi_df['B_tokenized'].apply(clean_text)

In [17]:
masnavi_df['M1'] =  masnavi_df['M1'].apply(lambda x:remove_new_line_char(normalizer.normalize(x)))

In [18]:
masnavi_df['M1_tokenized'] =  masnavi_df['M1'].apply(lambda x:hazm.word_tokenize(x))

In [19]:
masnavi_df['M1_tokenized'] = masnavi_df['M1_tokenized'].apply(clean_text)

In [20]:
masnavi_df['M2'] =  masnavi_df['M2'].apply(lambda x:remove_new_line_char(normalizer.normalize(x)))

In [21]:
masnavi_df['M2_tokenized'] =  masnavi_df['M2'].apply(lambda x:hazm.word_tokenize(x))

In [22]:
masnavi_df['M2_tokenized'] = masnavi_df['M2_tokenized'].apply(clean_text)

In [23]:
masnavi_df['B_clean'] = masnavi_df['B_tokenized'].apply(lambda x: " ".join(x))

In [24]:
masnavi_df.B_clean[200:250]

200        خر بهر دفع خار سوز درد جفته می‌انداخت صد زخم
201                       لگد کی دفع خار حاذقی مرکز تند
202                       جهد خار محکمتر زند عاقلی خاری
203                 حکیم خارچین استاد دست میزد می‌آزمود
204                   کنیزک طریق داستان می‌پرسید دوستان
205           حکیم رازها میگفت فاش مقام خواجگان شهر تاش
206           قصه گفتنش میداشت گوش نبض جستنش میداشت هوش
207                     نبض نام کی جهان مقصود جانش جهان
208                     دوستان شهر شمرد شهر دگر نام برد
209                           شدی شهر کدامین شهر میبودی
210                          نام شهری گذشت رنگ نبض نگشت
211                               خواجگان شهرها نان نمک
212    شهر شهر خانه خانه قصه نی رگش جنبید نی رخ گشت زرد
213                    نبض بد گزند بپرسید سمرقند چو قند
214            آه سردی برکشید ماه آب چشمش روان همچو جوی
215              بازرگانم آورید خواجه‌ای زرگر شهرم خرید
216                    ششماه فروخت بگفت زآتش غم برفروخت
217                        نبض جست سرخ زرد سمرقن

In [26]:
from itertools import chain
from collections import Counter

In [27]:
all_words = list(chain.from_iterable(masnavi_df.B_tokenized))

In [28]:
words_frequencies = nltk.FreqDist(all_words).most_common(100)

In [29]:
words_frequencies[:10]

[('اندر', 1467),
 ('جان', 1267),
 ('سر', 1008),
 ('حق', 955),
 ('دل', 861),
 ('آب', 787),
 ('کن', 718),
 ('نی', 715),
 ('کی', 671),
 ('صد', 663)]

In [30]:
print ('%-16s' % 'Number of words', '%-16s' % len(all_words))
print ('%-16s' % 'Number of unique words', '%-16s' % len(set(all_words)))
avg=np.sum([len(word) for word in all_words])/len(all_words)
print ('%-16s' % 'Average word length', '%-16s' % avg)
print ('%-16s' % 'Longest word', '%-16s' % all_words[np.argmax([len(word) for word in all_words])])

Number of words  164343          
Number of unique words 22429           
Average word length 3.9420906275290095
Longest word     نخواهم_هدیه‌ات  


In [31]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

In [32]:
cv=CountVectorizer()
word_count_vector = cv.fit_transform(all_words)

In [33]:
tfidf_transformer = TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [34]:
# print idf values 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names_out(),columns=["idf_weights"]) 
 
# sort ascending 
df_idf = df_idf.sort_values(by=['idf_weights'])

In [35]:
df_idf

,idf_weights
اندر,5.717380
می,5.862158
جان,5.863733
سر,6.092011
حق,6.141743
...,...
فرتوت,12.316570
بچید,12.316570
فرجۀ,12.316570
بچگانند,12.316570


In [36]:
masnavi_df[masnavi_df['']]

,Daftar,Poem,BNo,B,M1,M2,B_tokenized,M1_tokenized,M2_tokenized,B_clean
0,1,1,1,بشنو از نی، چون حکایت میکند واز جدائی‌ها شکایت...,بشنو از نی، چون حکایت میکند,واز جدائی‌ها شکایت میکند,"[بشنو, نی, حکایت, میکند, جدائی‌ها, شکایت, میکند]","[بشنو, نی, حکایت, میکند]","[جدائی‌ها, شکایت, میکند]",بشنو نی حکایت میکند جدائی‌ها شکایت میکند
1,1,1,2,کز نیستان تا مرا ببریده‌اند از نفیرم مرد و زن ...,کز نیستان تا مرا ببریده‌اند,از نفیرم مرد و زن نالیده‌اند,"[نیستان, ببریده‌اند, نفیرم, مرد, زن, نالیده‌اند]","[نیستان, ببریده‌اند]","[نفیرم, مرد, زن, نالیده‌اند]",نیستان ببریده‌اند نفیرم مرد زن نالیده‌اند
2,1,1,3,سینه خواهم شرحه شرحه از فراق تا بگویم شرح درد ...,سینه خواهم شرحه شرحه از فراق,تا بگویم شرح درد اشتیاق,"[سینه, خواهم_شرحه, شرحه, فراق, بگویم, شرح, درد...","[سینه, خواهم_شرحه, شرحه, فراق]","[بگویم, شرح, درد, اشتیاق]",سینه خواهم_شرحه شرحه فراق بگویم شرح درد اشتیاق
3,1,1,4,هر کسی کاو دور ماند از اصل خویش باز جوید روزگ...,هر کسی کاو دور ماند از اصل خویش,باز جوید روزگار وصل خویش,"[ماند, اصل, جوید, روزگار, وصل]","[ماند, اصل]","[جوید, روزگار, وصل]",ماند اصل جوید روزگار وصل
4,1,1,5,من به هر جمعیتی نالان شدم جفت بد حالان و خوش ح...,من به هر جمعیتی نالان شدم,جفت بد حالان و خوش حالان شدم,"[جمعیتی, نالان, شدم, جفت, بد, حالان, خوش, حالا...","[جمعیتی, نالان, شدم]","[جفت, بد, حالان, خوش, حالان, شدم]",جمعیتی نالان شدم جفت بد حالان خوش حالان شدم
...,...,...,...,...,...,...,...,...,...,...
22595,6,187,98,* تو عدوی، وز عدو شهد و لبن بی تکلف زهر گردد د...,* تو عدوی، وز عدو شهد و لبن,بی تکلف زهر گردد در دهن,"[عدوی, عدو, شهد, لبن, تکلف, زهر, دهن]","[عدوی, عدو, شهد, لبن]","[تکلف, زهر, دهن]",عدوی عدو شهد لبن تکلف زهر دهن
22596,6,187,99,هر وجودی کز عدم بنمود سر بر یکی زهر است و، بر ...,هر وجودی کز عدم بنمود سر,بر یکی زهر است و، بر دیگر شکر,"[وجودی, بنمود, سر, زهر, شکر]","[وجودی, بنمود, سر]","[زهر, شکر]",وجودی بنمود سر زهر شکر
22597,6,187,100,دوست شو، وز خوی ناخوش شو بری تا ز خمرۀ زهر هم ...,دوست شو، وز خوی ناخوش شو بری,تا ز خمرۀ زهر هم حلوا خوری,"[دوست, شو, خوی, ناخوش, شو, بری, خمرۀ, زهر, حلو...","[دوست, شو, خوی, ناخوش, شو, بری]","[خمرۀ, زهر, حلوا, خوری]",دوست شو خوی ناخوش شو بری خمرۀ زهر حلوا خوری
22598,6,187,101,ز آن نشد فاروق را زهری گزند که بد آن تریاق فار...,ز آن نشد فاروق را زهری گزند,که بد آن تریاق فاروقیش قند,"[نشد, فاروق, زهری, گزند, بد, تریاق, فاروقیش, قند]","[نشد, فاروق, زهری, گزند]","[بد, تریاق, فاروقیش, قند]",نشد فاروق زهری گزند بد تریاق فاروقیش قند


In [37]:
dp_groups = masnavi_df.groupby(['Daftar','Poem'])

In [38]:
for name, group in dp_groups:
    print(name)

(1, '1')
(1, '10')
(1, '100')
(1, '101')
(1, '102')
(1, '103')
(1, '104')
(1, '105')
(1, '106')
(1, '107')
(1, '108')
(1, '109')
(1, '11')
(1, '110')
(1, '111')
(1, '112')
(1, '113')
(1, '114')
(1, '115')
(1, '116')
(1, '117')
(1, '118')
(1, '119')
(1, '12')
(1, '120')
(1, '121')
(1, '122')
(1, '123')
(1, '124')
(1, '125')
(1, '126')
(1, '127')
(1, '128')
(1, '129')
(1, '13')
(1, '130')
(1, '131')
(1, '132')
(1, '133')
(1, '134')
(1, '135')
(1, '136')
(1, '137')
(1, '138')
(1, '139')
(1, '14')
(1, '140')
(1, '141')
(1, '142')
(1, '143')
(1, '144')
(1, '145')
(1, '146')
(1, '147')
(1, '148')
(1, '149')
(1, '15')
(1, '150')
(1, '151')
(1, '152')
(1, '153')
(1, '154')
(1, '155')
(1, '156')
(1, '157')
(1, '158')
(1, '159')
(1, '16')
(1, '160')
(1, '161')
(1, '162')
(1, '163')
(1, '164')
(1, '165')
(1, '166')
(1, '167')
(1, '168')
(1, '169')
(1, '17')
(1, '170')
(1, '171')
(1, '172')
(1, '173')
(1, '174')
(1, '175')
(1, '176')
(1, '177')
(1, '178')
(1, '179')
(1, '18')
(1, '180')
(1, '181')

In [ ]:
dictionary = gensim.corpora.Dictionary(masnavi_df.B_tokenized)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in masnavi_df.B_tokenized]

In [ ]:
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 8, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [ ]:
from gensim.test.utils import datapath

In [ ]:
lda_path = datapath("model")
lda_model.save(lda_path)

In [ ]:
lda = lda_model.load(lda_path)

In [ ]:
from pprint import pprint

In [ ]:
pprint(lda.print_topics())